Submitting data from Synapse scratch space

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import synapseclient
import cmc_submit2ndar as s2n
import pandas as pd
import numpy as np

In [2]:
syn = synapseclient.login()

Welcome, Attila Jones!



The data files to submit

In [9]:
fpath = '/home/attila/projects/bsm/results/2021-02-02-submit-to-nda/files_not_uploaded_yet.MSSM.txt'
dfiles = pd.read_csv(fpath, sep='\t', names=['synapseID', 'filename'])
s = dfiles['filename']
index = pd.MultiIndex.from_tuples(s.str.split('_NeuN_pl.').apply(lambda x: ['CMC_' + x[0], x[1]]), names=['indivID', 'filetype'])
dfiles = pd.DataFrame(dfiles.to_numpy(), index=index, columns=dfiles.columns).sort_index(axis=0, level='filetype')
ftypes = dfiles.index.get_level_values(1).unique()
print(ftypes)
dfiles

Index(['cram', 'cram.crai', 'flagstat.txt', 'ploidy_12.vcf.gz',
       'ploidy_12.vcf.gz.tbi', 'ploidy_2.vcf.gz', 'ploidy_2.vcf.gz.tbi',
       'ploidy_50.vcf.gz', 'ploidy_50.vcf.gz.tbi', 'unmapped.bam'],
      dtype='object', name='filetype')


,,synapseID,filename
indivID,filetype,,
CMC_MSSM_027,cram,syn22007834,MSSM_027_NeuN_pl.cram
CMC_MSSM_033,cram,syn22090557,MSSM_033_NeuN_pl.cram
CMC_MSSM_055,cram,syn22162964,MSSM_055_NeuN_pl.cram
CMC_MSSM_056,cram,syn22093909,MSSM_056_NeuN_pl.cram
CMC_MSSM_063,cram,syn22150192,MSSM_063_NeuN_pl.cram
...,...,...,...
CMC_PITT_072,unmapped.bam,syn22385266,PITT_072_NeuN_pl.unmapped.bam
CMC_PITT_082,unmapped.bam,syn22384272,PITT_082_NeuN_pl.unmapped.bam
CMC_PITT_098,unmapped.bam,syn22385231,PITT_098_NeuN_pl.unmapped.bam


In [4]:
prefixes = {'cram': '3338602',
            'cram.crai': '3338602',
            'flagstat.txt': '3340241',
            'ploidy_2.vcf.gz': '3340241',
            'ploidy_2.vcf.gz.tbi': '3340241',
            'ploidy_12.vcf.gz': '3340241',
            'ploidy_12.vcf.gz.tbi': '3340241',
            'ploidy_50.vcf.gz': '3340241',
            'ploidy_50.vcf.gz.tbi': '3340241',
            'unmapped.bam': '3338602'
           }
set(ftypes) == set(prefixes.keys())

True

In [5]:
basen = '/home/attila/projects/bsm/results/2021-02-02-submit-to-nda/genomics_sample03-'
fpaths = [basen + x + '.csv' for x in ['2019-12-09', '2020-06-08']]
oldgsam = pd.concat([pd.read_csv(x, skiprows=1) for x in fpaths], axis=0)
oldgsam

,subjectkey,experiment_id,src_subject_id,interview_age,interview_date,sample_description,sample_id_original,organism,sample_amount,sample_unit,...,patient_id_biorepository,sample_id_biorepository,cell_id_original,cell_id_biorepository,comments_misc,site,rat280,rat230,gqn,seq_batch
0,NDAR_INV0971H4H4,1223,CMC_MSSM_033,972,04/13/2018,frontal cortex,MSSM_033.DLPFC_1355.np1,human,1.0,NaN,...,CMC_MSSM_033,MSSM_033.DLPFC_1355.np1,NaN,NaN,NaN,U01MH106891,NaN,NaN,NaN,NaN
1,NDAR_INV0971H4H4,1223,CMC_MSSM_033,972,04/13/2018,frontal cortex,MSSM_033.DLPFC_1355.np1,human,1.0,NaN,...,CMC_MSSM_033,MSSM_033.DLPFC_1355.np1,NaN,NaN,NaN,U01MH106891,NaN,NaN,NaN,NaN
2,NDAR_INV1VPUF5CL,1223,CMC_MSSM_056,804,04/13/2018,frontal cortex,MSSM_056.DLPFC_1181.np1,human,1543.0,ng,...,CMC_MSSM_056,MSSM_056.DLPFC_1181.np1,NaN,NaN,NaN,U01MH106891,NaN,NaN,NaN,NaN
3,NDAR_INV1VPUF5CL,1223,CMC_MSSM_056,804,04/13/2018,frontal cortex,MSSM_056.DLPFC_1181.np1,human,1543.0,ng,...,CMC_MSSM_056,MSSM_056.DLPFC_1181.np1,NaN,NaN,NaN,U01MH106891,NaN,NaN,NaN,NaN
4,NDAR_INV1VPUF5CL,1223,CMC_MSSM_056,804,04/13/2018,frontal cortex,MSSM_056.DLPFC_1181.np1,human,1543.0,ng,...,CMC_MSSM_056,MSSM_056.DLPFC_1181.np1,NaN,NaN,NaN,U01MH106891,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,NDAR_INVTY432EB9,1223,CMC_MSSM_379,852,06/08/2020,frontal cortex,1238.np1,human,1.0,NaN,...,CMC_MSSM_379,1238.np1,NaN,NaN,NaN,U01MH106891,NaN,NaN,NaN,NaN
65,NDAR_INVLH141LX6,1223,CMC_MSSM_340,912,06/08/2020,frontal cortex,1247.np1,human,1.0,NaN,...,CMC_MSSM_340,1247.np1,NaN,NaN,NaN,U01MH106891,NaN,NaN,NaN,NaN
66,NDAR_INVAK206LGH,1223,CMC_MSSM_362,876,06/08/2020,frontal cortex,1346.np1,human,1.0,NaN,...,CMC_MSSM_362,1346.np1,NaN,NaN,NaN,U01MH106891,NaN,NaN,NaN,NaN
67,NDAR_INVTM018EUJ,1223,CMC_MSSM_405,708,06/08/2020,frontal cortex,1357.np1,human,1.0,NaN,...,CMC_MSSM_405,1357.np1,NaN,NaN,NaN,U01MH106891,NaN,NaN,NaN,NaN


In [6]:
gsam = s2n.edit_gsam(oldgsam)
gsam

,subjectkey,experiment_id,src_subject_id,interview_age,interview_date,sample_description,sample_id_original,organism,sample_amount,sample_unit,...,patient_id_biorepository,sample_id_biorepository,cell_id_original,cell_id_biorepository,comments_misc,site,rat280,rat230,gqn,seq_batch
src_subject_id,,,,,,,,,,,,,,,,,,,,,
CMC_MSSM_027,NDAR_INVYK598VAX,1223,CMC_MSSM_027,792,06/08/2020,frontal cortex,1421.np1,human,1.0,None,...,CMC_MSSM_027,1421.np1,NaN,NaN,NaN,U01MH106891,NaN,NaN,NaN,NaN
CMC_MSSM_033,NDAR_INV0971H4H4,1223,CMC_MSSM_033,972,04/13/2018,frontal cortex,MSSM_033.DLPFC_1355.np1,human,1.0,None,...,CMC_MSSM_033,MSSM_033.DLPFC_1355.np1,NaN,NaN,NaN,U01MH106891,NaN,NaN,NaN,NaN
CMC_MSSM_055,NDAR_INVKM569ZZK,1223,CMC_MSSM_055,792,06/08/2020,frontal cortex,1180.np1,human,1.0,None,...,CMC_MSSM_055,1180.np1,NaN,NaN,NaN,U01MH106891,NaN,NaN,NaN,NaN
CMC_MSSM_056,NDAR_INV1VPUF5CL,1223,CMC_MSSM_056,804,04/13/2018,frontal cortex,MSSM_056.DLPFC_1181.np1,human,1543.0,ng,...,CMC_MSSM_056,MSSM_056.DLPFC_1181.np1,NaN,NaN,NaN,U01MH106891,NaN,NaN,NaN,NaN
CMC_MSSM_063,NDAR_INVHG861GX7,1223,CMC_MSSM_063,864,06/08/2020,frontal cortex,CMC_MSSM_DLPC133.np1,human,1.0,None,...,CMC_MSSM_063,CMC_MSSM_DLPC133.np1,NaN,NaN,NaN,U01MH106891,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CMC_PITT_082,NDAR_INVGF082FCE,1223,CMC_PITT_082,732,06/08/2020,frontal cortex,551.np1,human,1.0,None,...,CMC_PITT_082,551.np1,NaN,NaN,NaN,U01MH106891,NaN,NaN,NaN,NaN
CMC_PITT_098,NDAR_INVTH969ZYH,1223,CMC_PITT_098,672,06/08/2020,frontal cortex,685.np1,human,1.0,None,...,CMC_PITT_098,685.np1,NaN,NaN,NaN,U01MH106891,NaN,NaN,NaN,NaN
CMC_PITT_101,NDAR_INVY3TCVYKD,1223,CMC_PITT_101,504,04/13/2018,frontal cortex,PITT_101.DRPC700.np1,human,408.0,ng,...,CMC_PITT_101,PITT_101.DRPC700.np1,NaN,NaN,NaN,U01MH106891,NaN,NaN,NaN,NaN


In [8]:
%connect_info

{
  "shell_port": 55573,
  "iopub_port": 53627,
  "stdin_port": 43305,
  "control_port": 33269,
  "hb_port": 55567,
  "ip": "127.0.0.1",
  "key": "792fd702-7d2d166517dd87e3dbdd3937",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-a45072be-0fe6-4354-aa97-006ce5122903.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
